In [3]:
#!pip install nltk
#!pip install unidecode

In [4]:
from encodings.utf_8 import encode
from time import sleep
import pandas as pd
import unidecode
import numpy as np
from matplotlib import pyplot as plt #para plotar os gráficos
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from selenium import webdriver
#from selenium.webdriver.firefox.service import Service
#from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re


texto = pd.read_csv("final3.txt", sep="\r")

for i in texto:
    print(i)


FileNotFoundError: File b'final3.txt' does not exist

In [ ]:
linhas = texto.values.tolist()

In [ ]:
print(linhas)

In [ ]:
postes = []
for dados in linhas:
    for posts in dados:
        posts = unidecode.unidecode(posts)
        posts = posts.lower().replace("... ver mais", "")
        postes.append(posts) 

In [ ]:
postes

In [ ]:
coun_vect = CountVectorizer()
count_matrix = coun_vect.fit_transform(postes)
count_array = count_matrix.toarray()
df = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names())
df

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(postes)
vectorizer.get_feature_names()
X_Array = X.toarray()
df1 = pd.DataFrame(data=X_Array,columns = vectorizer.get_feature_names())
df1
#df1.to_csv("TFIDF.csv", encoding = "utf-8", index = False)

In [ ]:
divisao = df1/df
divisao.fillna(0)

In [ ]:
dataset = X_Array

In [ ]:
# determining the maximum number of clusters 
# using the simple method
limit = int((dataset.shape[0]//2)**0.5)
  
# selecting optimal value of 'k'
# using elbow method
  
# wcss - within cluster sum of
# squared distances
wcss = {}
  
for k in range(2,limit+1):
    model = KMeans(n_clusters=k)
    model.fit(dataset)
    wcss[k] = model.inertia_      


In [ ]:
keys = wcss.keys() 

In [ ]:
values = wcss.values()

In [ ]:
# plotting the wcss values
# to find out the elbow value
plt.plot(keys, values, 'gs-')
plt.xlabel('Values of "k"')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 3, #numero de clusters
                
init = 'k-means++', n_init = 10, #algoritmo que define a posição dos clusters de maneira mais assertiva

max_iter = 300) #numero máximo de iterações

pred_y = kmeans.fit_predict(dataset)

plt.scatter(dataset[:,1], dataset[:,0], c = pred_y) #posicionamento dos eixos x e y
plt.xlim(-0.5, 0.5) #range do eixo x
plt.ylim(-0.5, 0.5) #range do eixo y
plt.grid() #função que desenha a grade no nosso gráfico
plt.scatter(kmeans.cluster_centers_[:,1],kmeans.cluster_centers_[:,0], s = 70, c = 'red') #posição de cada centroide no gráfico
plt.show()

In [ ]:
kmeans.labels_

In [ ]:
df["cluster"] = kmeans.labels_

In [ ]:
df.columns

In [ ]:
df

In [ ]:
#df.to_csv("Postagens TFIDF.csv", encoding = "utf-8", index = False)

In [ ]:
dfposts = pd.DataFrame(postes, columns=["Posts"])


In [ ]:
dfposts["cluster"] = kmeans.labels_
dfposts

In [ ]:
dfposts[dfposts.cluster == 2].sample(5).values

In [ ]:
condicoes = [(dfposts['cluster'] == 0), 
             (dfposts['cluster'] == 1), 
             (dfposts['cluster'] == 2)]
opcoes = ["negativo", "neutro", "positivo"]
dfposts['sentimento'] = np.select(condicoes, opcoes)

In [ ]:
df["sentimento"] = dfposts['sentimento']
df1["sentimento"] = dfposts['sentimento']
divisao["sentimento"] = dfposts['sentimento']

In [ ]:
#dfposts.to_csv("Postagens com rotulo.csv", encoding = "utf-8", index = False)